## Prompting Gemini 2.5 Pro to create emails

### Importing libraries and key

In [1]:
import pandas as pd
import google.generativeai as genai
import json
import time
import os
from typing import List, Dict
import asyncio
from dotenv import load_dotenv
import ast 

# Load environment variables
load_dotenv()

# Configure the API
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
if not GEMINI_API_KEY:
    raise ValueError("Please set GEMINI_API_KEY in your .env file")

genai.configure(api_key=GEMINI_API_KEY)

# Load the recipe data
recipes_df = pd.read_csv('email_recipes_1250.csv')
print(f"Loaded {len(recipes_df)} recipes")
print("Sample recipe:")
print(recipes_df.iloc[0])

Loaded 1250 recipes
Sample recipe:
recipe_id                                                                        rID_262
length                                                                            Medium
sentiment                                                                  Very Negative
topics                                                            ['Housing & Planning']
persona_name                                     The 'Civic Pragmatist' Community Leader
characteristics.age                                                                   45
characteristics.gender_identity                                                   Female
characteristics.occupation                            Charity Manager / Community Leader
characteristics.political_leaning      Politically centrist, pragmatic, and non-parti...
characteristics.communication_style    Collaborative, solution-oriented, and professi...
characteristics.core_motivation        Improving the local community and se

### Creating email prompt

In [2]:
def create_email_prompt(recipe_row):
    """Create a detailed prompt for email generation based on recipe"""
    
    # Convert topics list from string back to list if needed
    topics = ast.literal_eval(recipe_row['topics']) if isinstance(recipe_row['topics'], str) else recipe_row['topics']
    topics_str = ", ".join(topics)
    
    prompt = f"""
You are tasked with generating a realistic email from a UK constituent to their Member of Parliament.

RECIPE PARAMETERS:
- Persona: {recipe_row['persona_name']}
- Age: {recipe_row['characteristics.age']}
- Gender: {recipe_row['characteristics.gender_identity']}
- Occupation: {recipe_row['characteristics.occupation']}
- Political Leaning: {recipe_row['characteristics.political_leaning']}
- Communication Style: {recipe_row['characteristics.communication_style']}
- Core Motivation: {recipe_row['characteristics.core_motivation']}
- Sentiment: {recipe_row['sentiment']}
- Length: {recipe_row['length']}
- Topics: {topics_str}

INSTRUCTIONS:
1. Generate a realistic email that this persona would write to their MP
2. The email should cover the specified topic(s) with the given sentiment
3. Match the communication style and motivation of the persona exactly
4. Length guidelines: Short=1-3 sentences, Medium=2-4 paragraphs, Long=5+ paragraphs
5. Include realistic UK references, place names, and political context
6. Make it feel authentic - include personality quirks, writing style variations
7. BE CREATIVE AND DIVERSE - vary writing styles, approaches, and specific examples

REQUIRED OUTPUT FORMAT (JSON):
{{
    "email_header": "Subject: [realistic subject line]",
    "email_body": "[full email body text]"
}}
"""
    return prompt

# Test with first recipe
test_prompt = create_email_prompt(recipes_df.iloc[0])
print("Test prompt created. Length:", len(test_prompt))

Test prompt created. Length: 1280


### Running test on single prompt 2.5 Pro

In [3]:
# Cell 1: Test Connection with Single Prompt on Gemini 2.5 Pro
def test_single_generation():
    """Test a single generation to verify API works with Gemini 2.5 Pro"""
    
    print("🧪 Testing single generation with Gemini 2.5 Pro...")
    
    # Use Gemini 2.5 Pro model
    model = genai.GenerativeModel('gemini-2.5-pro')
    
    # Create test prompt with first recipe
    test_prompt = create_email_prompt(recipes_df.iloc[0])
    
    try:
        print("📤 Sending test request...")
        start_time = time.time()
        
        response = model.generate_content(
            test_prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=1.4,  # High temperature for diversity
                max_output_tokens=1300,
                response_mime_type="application/json"
            )
        )
        
        end_time = time.time()
        
        print(f"✅ API Test Successful! (took {end_time - start_time:.2f} seconds)")
        print(f"Response length: {len(response.text)} characters")
        
        # Try to parse JSON
        result = json.loads(response.text)
        print(f"✅ JSON Parse Successful!")
        print(f"Recipe ID: {result.get('recipe_id')}")
        print(f"Subject: {result.get('email_header')}")
        print(f"Body length: {len(result.get('email_body', ''))} characters")
        
        print("\n" + "="*50)
        print("SAMPLE GENERATED EMAIL:")
        print("="*50)
        print(f"Subject: {result.get('email_header')}")
        print(f"Body:\n{result.get('email_body')}")
        print("="*50)
        
        return True, result
        
    except Exception as e:
        print(f"❌ Test failed: {e}")
        return False, None

# Run the test
test_run = False
if test_run:
    success, sample_result = test_single_generation()



In [11]:
from tqdm import tqdm
import time
import signal

def run_all_enhanced(recipes_df, batch_size=50, resume_from=0, verbose=True):
    """
    Enhanced version with rate limiting, progress tracking, and resume capability.
    Returns only original recipe columns + subject_line + email_body.
    """
    model = genai.GenerativeModel('gemini-2.5-flash')
    results_list = []
    
    # Resume from checkpoint if provided
    start_idx = resume_from
    
    # Create progress bar
    pbar = tqdm(total=len(recipes_df), initial=start_idx, desc="Generating emails")
    
    for index, recipe_row in recipes_df.iloc[start_idx:].iterrows():
        try:
            if verbose:
                print(f"\n🔄 Processing row {index}: {recipe_row['persona_name']}")
            
            test_prompt = create_email_prompt(recipe_row)
            if verbose:
                print(f"📝 Prompt created, sending to API...")
            
            # Add timeout and better error handling
            response = model.generate_content(
                test_prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=1.4,
                    max_output_tokens=3500,  # Reduced from 2000
                    response_mime_type="application/json"
                )
            )
            
            print(f"📨 Response received")
            
            # Check if response is valid
            if not response.candidates or not response.candidates[0].content:
                raise Exception("Empty response from API")
            
            candidate = response.candidates[0]
            if candidate.finish_reason != 1:  # Not STOP (normal completion)
                raise Exception(f"API response finish_reason: {candidate.finish_reason}")
            
            # Safe access to response text
            response_text = candidate.content.parts[0].text
            print(f"✅ Response text length: {len(response_text)}")
            
            generated_data = json.loads(response_text)
            
            # Only extract subject_line and email_body from generated content
            combined_data = {
                **recipe_row.to_dict(),
                'subject_line': generated_data.get('subject_line', ''),
                'email_body': generated_data.get('email_body', '')
            }
            results_list.append(combined_data)
            
            pbar.update(1)
            if verbose:
                print(f"✅ Row {index} completed successfully")
            
            # Rate limiting - pause between requests
            time.sleep(0.5)
            
            # Save checkpoint every batch_size items
            if len(results_list) % batch_size == 0:
                checkpoint_df = pd.DataFrame(results_list)
                checkpoint_df.to_csv(f'email_dataset_checkpoint_{index}.csv', index=False)
                print(f"💾 Checkpoint saved at row {index}")
                
        except KeyboardInterrupt:
            print(f"\n⏹️  Process interrupted by user at row {index}")
            break
            
        except Exception as e:
            print(f"❌ Failed at row {index}: {str(e)}")
            # Log failed row with empty email fields
            failed_data = {
                **recipe_row.to_dict(),
                'subject_line': '',
                'email_body': f'GENERATION_FAILED: {str(e)}'
            }
            results_list.append(failed_data)
            pbar.update(1)
            
            # Continue with next row instead of stopping
            continue
    
    pbar.close()
    
    # Create final DataFrame
    return_df = pd.DataFrame(results_list)
    return return_df


In [12]:
# Test with just 1 row first
print("🧪 Testing with single row...")
test_df = recipes_df.head(1)
result_df = run_all_enhanced(test_df, batch_size=1, resume_from=0, verbose=False)

if len(result_df) > 0 and result_df.iloc[0]['email_body'] != '' and not result_df.iloc[0]['email_body'].startswith('GENERATION_FAILED'):
    print("✅ Single row test successful! Running full dataset...")
    # Run full dataset
    result_df = run_all_enhanced(recipes_df, batch_size=50, resume_from=0, verbose=False)
    
    # Save final results
    result_df.to_csv('email_dataset_final.csv', index=False)
    
    # Print summary statistics (fixed)
    successful = len(result_df[~result_df['email_body'].str.startswith('GENERATION_FAILED', na=False)])
    total = len(result_df)
    print(f"\n📊 Generation Complete: {successful}/{total} emails generated successfully")
    print(f"Success rate: {successful/total*100:.1f}%")
else:
    print("❌ Single row test failed. Check the output above for errors.")
    print("First row result:")
    print(result_df.iloc[0] if len(result_df) > 0 else "No results")

🧪 Testing with single row...


Generating emails: 100%|██████████| 1/1 [00:14<00:00, 14.09s/it]

📨 Response received
✅ Response text length: 2775


Generating emails: 100%|██████████| 1/1 [00:14<00:00, 14.60s/it]


💾 Checkpoint saved at row 0
✅ Single row test successful! Running full dataset...


...

📨 Response received
❌ Failed at row 1248: API response finish_reason: 2


Generating emails: 100%|██████████| 1250/1250 [5:44:30<00:00, 14.27s/it]

📨 Response received
✅ Response text length: 1488


Generating emails: 100%|██████████| 1250/1250 [5:44:30<00:00, 16.54s/it]

💾 Checkpoint saved at row 1249

📊 Generation Complete: 1181/1250 emails generated successfully
Success rate: 94.5%
